In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re, time,csv
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json

outfile=open('output.json','w')

In [ ]:
def write_row(results):
    
    res = []
    for result in results:
        try:
            interview_title_box = result.find_element(by=By.CSS_SELECTOR, value='h2.mt-0.mb-xxsm.css-93svrw.el6ke055')
            interview_title = interview_title_box.text
        except NoSuchElementException as e:
            print('Could not extract Interview Title')
        
        try:
            date_box = result.find_element(by=By.CSS_SELECTOR, value='time')
            date = date_box.text
        except NoSuchElementException as e:
            print('Could not extract Interview Title')
        
        arr = []
        try:
            arr=result.find_elements(by=By.CSS_SELECTOR,value='span.mb-xxsm')
        except:
            print("Could not get the Offer, Experience and Interview ratings")
        
        try:
            offer_acceptance = arr[0].text
            experience_rating = arr[1].text
            interview_rating = arr[2].text
        except:
            print('Could not add anything')

        try:
            interview_process_block = result.find_elements(by=By.CSS_SELECTOR, value='p')
            interview_process = interview_process_block[3].text
        except:
            print("Could not get interview process text")

    
        dictionary = {
            "interview_title": interview_title,
            "date": date,
            "offer_acceptance": offer_acceptance,
            "experience_rating": experience_rating,
            "interview_rating": interview_rating,
            "interview_process": interview_process
        }
        res.append(dictionary)
    return res

        

In [ ]:
# defining credentials
email = "Enter email here"
password = "Enter password here"

# setting up  google chrome driver
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=/Users/parkashheerani/Library/Application Support/Google/Chrome/Default")
driver = webdriver.Chrome(options=options)

# going to microsoft page
time.sleep(1)
url = 'https://www.glassdoor.com/Interview/Microsoft-Interview-Questions-E1651_P1.htm'
driver.get(url)

# clicking on login page
time.sleep(2)
try:
    driver.find_element(by=By.CSS_SELECTOR, value = "div[class='d-none d-lg-block']").click()
except:
    pass

# Log in to Glassdoor

try:
    driver.find_element('name', 'username')
    email_input = driver.find_element('name', "username")
    email_input.send_keys(email)
    driver.find_element("name", 'submit').click()
    time.sleep(5)

    password_input = driver.find_element('name', "password")
    password_input.send_keys(password)
    driver.find_element("name", 'submit').click()
except:
    pass
# and the work begings


orig_url = url.partition("_")[0]

dictionary_list = []
 # visit the page

page_cnt=1 # keep track of page count

while True: # keep going until there are no more pages
    
    try:
        driver.get(orig_url +"_P" + str(page_cnt) + ".htm")
        if page_cnt != 1:
            time.sleep(5)
        print('page',page_cnt) # print current page count

         # increment 

        #extract and write the links from the current page
        results=driver.find_elements(by=By.CSS_SELECTOR,value='div.mt-0.mb-0.my-md-std.p-std.gd-ui-module.css-cup1a5.ec4dwm00')
        dictionary_list = dictionary_list + write_row(results)

        #wait until the next button appears   
                                                                   
        next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.nextButton.css-1iiwzeb.e13qs2072')))
        page_cnt+=1

        with open("output.json", "w") as outfile:
            json.dump(dictionary_list, outfile)
            if len(dictionary_list) > 1:
                print('Successfully Added')
            else:
                print('Couldnt add')
                print(dictionary_list)
            

    except:
        # Log in to Glassdoor
        try:
            driver.find_element('name', 'username')
            email_input = driver.find_element('name', "username")
            email_input.send_keys(email)
            driver.find_element("name", 'submit').click()
            time.sleep(5)

            password_input = driver.find_element('name', "password")
            password_input.send_keys(password)
            driver.find_element("name", 'submit').click()
        except:
            print('No more pages')
            continue
    if page_cnt > 1300:
        break
        

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('output.json')

In [ ]:
df.shape

In [ ]:
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.to_csv('data.csv')